In [ ]:
!pip install -q "transformers>=4.51.0" accelerate bitsandbytes sentencepiece

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Qwen3-8B
model_name = "Qwen/Qwen3-14B"

# ▼ 4bit 量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                         # ★ 4bit 量子化
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
    bnb_4bit_use_double_quant=True,           # さらに省メモリ
    bnb_4bit_quant_type="nf4",                # よく使われる方式
)

# tokenizer & model をロード
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",            # GPU があれば自動で使う
    quantization_config=bnb_config,
)


In [ ]:
import re

def summarize_qwen3(text: str, target_chars: int = 120) -> str:
    """
    Qwen3 を使って日本語要約する（英語禁止、思考モードOFF）。
    """

    user_prompt = (
        f"次の文章を、日本語で、だいたい {target_chars} 文字程度に自然に要約してください。\n"
        f"・出力は必ず日本語のみ\n"
        f"・英語や推論過程（<think>など）を出力しない\n"
        f"・結果だけ簡潔に\n\n"
        f"【文章】\n{text}\n\n【要約】"
    )

    messages = [
        {
            "role": "system",
            "content": (
                "あなたは日本語専用の要約アシスタントです。"
                "英語や他の言語、推論過程・自己コメントを絶対に出力してはいけません。"
                "回答は【要約】の後の文章のみとします。"
            ),
        },
        {"role": "user", "content": user_prompt},
    ]

    chat_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,

        assistant_response=False,
        enable_thinking=False,
        response_format="text",
    )

    inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.3,  # 少し下げて暴れにくく
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

    gen_ids = generated[0][inputs["input_ids"].shape[1]:]
    output = tokenizer.decode(gen_ids, skip_special_tokens=True)

    # 余計な部分があったらカット
    if "【要約】" in output:
        output = output.split("【要約】", 1)[-1].strip()

    # <think> や英語が誤って出ても除去
    output = re.sub(r"<think>.*?</think>", "", output, flags=re.DOTALL).strip()
    output = re.sub(r"[a-zA-Z]+", "", output).strip()

    return output


In [ ]:
text = input("要約したい文章を貼ってください：\n")
target = int(input("だいたい何文字くらいで要約してほしいですか？："))

print("\n--- Qwen3-8B 要約結果 ---\n")
summary = summarize_qwen3(text, target_chars=target)
print(summary)
print("\n【実際の文字数】：", len(summary))
